# Analyzing the Linear Relaxation

In [ ]:
from _evaluation import (  # the code has been moved to a separate file
    get_sample,
    n,
    samples,
)

## Getting a sample solution

The following code will create a random instance and solve

1. The linear relaxation with `>=2` subtour elimination constraints
2. The linear relaxation with `>=1` subtour elimination constraints
3. The integer programming problem to get an optimal solution

It will plot the solutions for you to compare and save the results in the `samples` list.

In [ ]:
# Run me to get a sample on linear relaxation and integral solution
get_sample()
print(f"There are now {len(samples)} samples")

In [ ]:
for i, (objective_value, linear_relaxation_value, overlap) in enumerate(samples):
    print(
        f"Sample {i}: Objective value: {objective_value}, Linear relaxation value (>=2): {linear_relaxation_value}, Overlap: {overlap}/{n}"
    )